In [16]:
import tensorflow as tf
import numpy as np 
import os 

SEED = 1234
tf.random.set_seed(SEED)
np.random.seed(SEED)

working_directory = os.getcwd()

In [18]:
from google.colab import drive
drive.mount(os.path.join(working_directory,'drive'))

Mounted at /content/drive


In [23]:
!ls drive/MyDrive/Colab\ Notebooks

 CNN_Classification.ipynb      MulticlassSegmentation.ipynb    unet.ipynb
'Copy of Unet.ipynb'	       OverfittingAndCallbacks.ipynb   Untitled
 Development_Dataset.zip       prova.ipynb		       Untitled0.ipynb
 Homework2_with_tiling.ipynb  'Segmentation (1).ipynb'	       Untitled1.ipynb
 Localization.ipynb	       SegmentationDataset.zip	       Untitled2.ipynb
 LSTM.ipynb		       Segmentation.ipynb


In [24]:
!unzip drive/MyDrive/Colab\ Notebooks/Development_Dataset.zip

Archive:  drive/MyDrive/Colab Notebooks/Development_Dataset.zip
  inflating: Development_Dataset/LICENSE.txt  
   creating: Development_Dataset/Test_Dev/
   creating: Development_Dataset/Test_Dev/Bipbip/
   creating: Development_Dataset/Test_Dev/Bipbip/Haricot/
   creating: Development_Dataset/Test_Dev/Bipbip/Haricot/Images/
  inflating: Development_Dataset/Test_Dev/Bipbip/Haricot/Images/Bipbip_haricot_im_00211.jpg  
  inflating: Development_Dataset/Test_Dev/Bipbip/Haricot/Images/Bipbip_haricot_im_00581.jpg  
  inflating: Development_Dataset/Test_Dev/Bipbip/Haricot/Images/Bipbip_haricot_im_00721.jpg  
  inflating: Development_Dataset/Test_Dev/Bipbip/Haricot/Images/Bipbip_haricot_im_00951.jpg  
  inflating: Development_Dataset/Test_Dev/Bipbip/Haricot/Images/Bipbip_haricot_im_01341.jpg  
  inflating: Development_Dataset/Test_Dev/Bipbip/Haricot/Images/Bipbip_haricot_im_02421.jpg  
  inflating: Development_Dataset/Test_Dev/Bipbip/Haricot/Images/Bipbip_haricot_im_02781.jpg  
  inflating: De

In [54]:
# Dataset is now in the working_directory
# folder to save the images and masks for training
import shutil 
from tqdm.notebook import tqdm
from time import sleep

images_directory_t = os.path.join(working_directory,'images')
masks_directory_t = os.path.join(working_directory,'masks')

if not os.path.exists(images_directory_t):
  os.mkdir(images_directory_t)
if not os.path.exists(masks_directory_t):
  os.mkdir(masks_directory_t)

dataset_dir_t = os.path.join(working_directory,'Development_Dataset/Training') 
datasets = os.listdir(dataset_dir_t)
plants = ['Haricot','Mais']

image_names = []
mask_names = []

count = 1
for d in datasets:
  print("[COPYING] dataset ",count,"/",len(datasets))
  
  for p in plants:
    image_names = os.listdir(os.path.join(dataset_dir_t,d,p,'Images'))
    mask_names = os.listdir(os.path.join(dataset_dir_t,d,p,'Masks'))
    
    for i in tqdm(range(0,len(image_names))):
      sleep(0.01)
    
    # Source - destination
    for i in image_names:
      shutil.copy(os.path.join(dataset_dir_t,d,p,'Images',i),images_directory_t)
    for m in mask_names:
      shutil.copy(os.path.join(dataset_dir_t,d,p,'Masks',m),masks_directory_t)
  
  count += 1



[COPYING] dataset  1 / 4



[COPYING] dataset  2 / 4



[COPYING] dataset  3 / 4



[COPYING] dataset  4 / 4


In [57]:
# tile dirs 

images_tiles = os.path.join(images_directory_t,'tiles')
masks_tiles = os.path.join(masks_directory_t,'tiles')

if not os.path.exists(images_tiles):
  os.mkdir(images_tiles)
if not os.path.exists(masks_tiles):
  os.mkdir(masks_tiles)


In [ ]:
from math import ceil

def tile(image_array,tile_size,overlapping_factor):
    q_overlap = int(tile_size/overlapping_factor)
    
    original_shape = image_array.shape
    padded_shape = [ceil(image_array.shape[0]/tile_size)*tile_size,ceil(image_array.shape[1]/tile_size)*tile_size,image_array.shape[2]]
    padded_image = np.array(np.zeros(padded_shape,dtype = np.uint8))
    padded_image[:image_array.shape[0],:image_array.shape[1],:image_array.shape[2]] = image_array
    
    x = []
    y = []

    for i in range(0,padded_shape[0],tile_size-q_overlap):
        x.append(i)
    for i in range(0,padded_shape[1],tile_size-q_overlap):
        y.append(i)
  
    tiles = []
    for i in x:
        for j in y:
            t_tile = np.array(np.zeros([tile_size,tile_size,padded_shape[2]]))
            t_tile = padded_image[i:i+tile_size,j:j+tile_size,:] 
            tiles.append(t_tile)
    
    return tiles

def untile(tiles,tile_size,original_shape,overlapping_factor):
    
    q_overlap = int(tile_size/overlapping_factor)
    
    target_image = np.array(np.zeros(original_shape,dtype=np.uint8))
    
    padded_shape = [ceil(original_shape[0]/tile_size)*tile_size,ceil(original_shape[1]/tile_size)*tile_size,original_shape[2]]
    padded_image = np.array(np.zeros(padded_shape,dtype = np.uint8))
    
    x = []
    y = []
    
    for i in range(0,padded_shape[0],tile_size-q_overlap):
        x.append(i)
    for i in range(0,padded_shape[1],tile_size-q_overlap):
        y.append(i)
    
    count = 0
    for i in x:
        for j in y:
            padded_image[i:tile_size+i,j:j+tile_size,:] = tiles[count] 
            count += 1
    
    target_image = padded_image[:original_shape[0],:original_shape[1],:]
    print(target_image.shape)
    return target_image
            



tile_size = 512
overlapping_factor = 8


# tile creation for both masks and images 
from PIL import Image

images = os.listdir(images_directory_t) 
masks = os.listdir(masks_directory_t)
images.remove('tiles')
masks.remove('tiles')

k = 1
  
for i in images:
  print("[TILING-images] ",k,"/",len(images))
  tiles = []
  img = Image.open(os.path.join(images_directory_t,i))
  img_array = np.array(img)
  tiles = tile(img_array,tile_size,overlapping_factor)

  count = 0
  print("[SAVING] tiles")
  for t in tiles:
    
    tile_image = Image.fromarray(t,'RGB')
    tile_name = str(count) + '_' + i 
    tile_image = tile_image.save(os.path.join(images_directory_t,'tiles',tile_name))
    count += 1
  k += 1

k = 0

for m in masks:
  
  print("[TILING-masks] ",k,"/",len(masks))  
  tiles = []
  img = Image.open(os.path.join(masks_directory_t,m))
  img_array = np.array(img)
  tiles = tile(img_array,tile_size,overlapping_factor)

  count = 0
  print("[SAVING] tiles")
  for t in tiles:
    tile_image = Image.fromarray(t,'RGB')
    tile_name = str(count) + '_' + m
    tile_image = tile_image.save(os.path.join(masks_directory_t,'tiles',tile_name))
    count += 1
  
  k += 1

In [ ]:
!ls images/tiles

In [78]:
img_training_directory = os.path.join(images_directory_t,'tiles')
msk_trianing_directory = os.path.join(masks_directory_t,'tiles')
